In [79]:
pip uninstall pulp

Found existing installation: PuLP 3.1.1
Uninstalling PuLP-3.1.1:
  Would remove:
    /usr/local/bin/pulptest
    /usr/local/lib/python3.11/dist-packages/pulp-3.1.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/pulp/*
Proceed (Y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/uninstall.py", line 106, in run
    uninstall_pathset = req.uninstall(
                        ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/req/req_install.py", line 722, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/req/req_uninstall.py", line 364, in remove
    if auto_confirm or self._allowed_to_proceed(verbose):
         

In [80]:
pip install pulp

In [81]:
import pandas as pd
from pulp import *

In [82]:
#Create a variable to hold the problem data
problem = LpProblem("Cost_Min", LpMinimize)

In [83]:
# Define the supply and demand values
# There are 3 Factories: 1 in Bregenz, 1 in Zürich and 1 in Munich, Each Factory has different supply cacpacity.
supply = {
    "Bregenz": 100,
    "Zürich": 150,
    "Munich": 200
}

# There are 5 Retailers, Each Retailer has different demand.
demand = {
    "Konstanz": 80,
    "Friedrichshafen": 50,
    "Buchs": 70,
    "Lindau": 60,
    "Lochau": 120
}

In [84]:
# Create a list of sources and destinations
sources = list(supply.keys())
destinations = list(demand.keys())

In [85]:
# Define the transportation cost matrix using nested lists
costs = [
    [1000, 900, 900, 300, 300],  # Transportation Costs from Bregenz to: Konstanz, Friedrichshafen, Buchs, Lindau, Lochau
    [900, 1100, 400, 900, 1000], # Transportation Costs from Zürich to: Konstanz, Friedrichshafen, Buchs, Lindau, Lochau
    [500, 500, 900, 500, 700]   # Transportation Costs from Munich to: Konstanz, Friedrichshafen, Buchs, Lindau, Lochau
]

# Create the cost dictionary using the given costs matrix
cost = {
    (sources[i], destinations[j]): costs[i][j]
    for i in range(len(sources))
    for j in range(len(destinations))
}

In [86]:
# Create binary decision variables for each source-destination pair
X = LpVariable.dicts("X", (sources, destinations), lowBound=0, cat=LpInteger)

In [87]:
# Define the objective function
problem += pulp.lpSum(X[source][dest] * cost[source, dest] for source in sources for dest in destinations), "Total Cost"

In [88]:
#Add supply constraints
for source in sources:
    problem += pulp.lpSum(X[source][dest] for dest in destinations) <= supply[source], f"Supply{source}"

In [89]:
#Add demand constraints
for dest in destinations:
    problem += pulp.lpSum(X[source][dest] for source in sources) == demand[dest], f"Demand{dest}"

In [90]:
#Solve the problem
problem.solve()

1

In [92]:
# Optimized allocation: total transportation cost is EUR 170 000
# Cost-effective and balanced distribution: satisfies both the supply capacity of each factory and the demand of each retailer
for source in sources:
    for dest in destinations:
        if X[source][dest].varValue > 0:
            print(f"Ship {X[source][dest].varValue} units from {source} to {dest}")

print(f"Total Cost: EUR {pulp.value(problem.objective)}")

Ship 100.0 units from Bregenz to Lochau
Ship 70.0 units from Zürich to Buchs
Ship 10.0 units from Zürich to Lochau
Ship 80.0 units from Munich to Konstanz
Ship 50.0 units from Munich to Friedrichshafen
Ship 60.0 units from Munich to Lindau
Ship 10.0 units from Munich to Lochau
Total Cost: EUR 170000.0


In [ ]:
# https://coin-or.github.io/pulp/